In [1]:
import urllib
import pandas as pd
import moment
from datetime import datetime, timedelta
import pendulum
import json
import requests

In [2]:
url='https://gbfs.divvybikes.com/gbfs/en/station_information.json'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
result = requests.get(url,headers= headers)
result = result.json()

In [18]:
result['data']['stations']

[{'station_id': '2',
  'eightd_station_services': [],
  'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
   'ios': 'https://chi.lft.to/lastmile_qr_scan'},
  'lat': 41.87651122881695,
  'lon': -87.62054800987242,
  'station_type': 'classic',
  'has_kiosk': True,
  'name': 'Buckingham Fountain',
  'electric_bike_surcharge_waiver': False,
  'rental_methods': ['TRANSITCARD', 'CREDITCARD', 'KEY'],
  'capacity': 39,
  'external_id': 'a3a36d9e-a135-11e9-9cda-0a87ae2ba916',
  'short_name': '15541',
  'eightd_has_key_dispenser': False,
  'legacy_id': '2'},
 {'station_id': '3',
  'eightd_station_services': [],
  'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
   'ios': 'https://chi.lft.to/lastmile_qr_scan'},
  'lat': 41.86722595682,
  'lon': -87.6153553902,
  'station_type': 'classic',
  'has_kiosk': True,
  'name': 'Shedd Aquarium',
  'electric_bike_surcharge_waiver': False,
  'rental_methods': ['TRANSITCARD', 'CREDITCARD', 'KEY'],
  'capacity': 55,
  'externa

In [31]:
stations = pd.DataFrame(result['data']['stations'])[['station_id','lat','lon','name']]

In [32]:
df = pd.read_csv('./2020/Divvy_Trips_2020_Q1.csv')
df.drop(columns=['ride_id','rideable_type','member_casual'],inplace=True)
df.head()

,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng
0,1/1/2020 0:04,1/1/2020 0:17,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246.0,41.9363,-87.6527,41.9561,-87.6688
1,1/1/2020 0:10,1/1/2020 0:10,Daley Center Plaza,81,Daley Center Plaza,81.0,41.8842,-87.6296,41.8842,-87.6296
2,1/1/2020 0:11,1/1/2020 0:15,Daley Center Plaza,81,Dearborn St & Van Buren St,624.0,41.8842,-87.6296,41.8763,-87.6292
3,1/1/2020 0:11,1/1/2020 0:13,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256.0,41.9542,-87.6544,41.9528,-87.6500
4,1/1/2020 0:12,1/1/2020 0:14,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59.0,41.8675,-87.6322,41.8672,-87.6260


In [34]:
df=df[df['started_at'] != df['ended_at']]
df=df[df['start_station_name'] != 'HQ QR']
df.shape

(420620, 10)

In [35]:
df.rename(columns={"start_station_id": "from_station_id", "end_station_id": "to_station_id","started_at":"start_time",
                   "ended_at" : "end_time"
                  },inplace=True)
df.head()

,start_time,end_time,start_station_name,from_station_id,end_station_name,to_station_id,start_lat,start_lng,end_lat,end_lng
0,1/1/2020 0:04,1/1/2020 0:17,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246.0,41.9363,-87.6527,41.9561,-87.6688
2,1/1/2020 0:11,1/1/2020 0:15,Daley Center Plaza,81,Dearborn St & Van Buren St,624.0,41.8842,-87.6296,41.8763,-87.6292
3,1/1/2020 0:11,1/1/2020 0:13,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256.0,41.9542,-87.6544,41.9528,-87.6500
4,1/1/2020 0:12,1/1/2020 0:14,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59.0,41.8675,-87.6322,41.8672,-87.6260
5,1/1/2020 0:19,1/1/2020 0:26,Clark St & Leland Ave,326,Southport Ave & Irving Park Rd,318.0,41.9671,-87.6674,41.9542,-87.6644


In [ ]:
df.set_index('key').join(other.set_index('key'))

In [5]:
import numpy as np
df['start'] = pd.to_datetime(df['start_time'])
df['start'] =df['start'].values.astype(np.int64) // 10**6

In [5]:
 df['start'] = df.apply(lambda row :pendulum.from_format(row['start_time'], "MM/DD/YYYY HH:mm",tz='America/Chicago').timestamp()*1000,axis = 1) 

In [6]:
df.head()

,start_time,end_time,start_station_name,from_station_id,end_station_name,to_station_id,start_lat,start_lng,end_lat,end_lng,start
0,1/1/2020 0:04,1/1/2020 0:17,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246.0,41.9363,-87.6527,41.9561,-87.6688,1.577859e+12
2,1/1/2020 0:11,1/1/2020 0:15,Daley Center Plaza,81,Dearborn St & Van Buren St,624.0,41.8842,-87.6296,41.8763,-87.6292,1.577859e+12
3,1/1/2020 0:11,1/1/2020 0:13,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256.0,41.9542,-87.6544,41.9528,-87.6500,1.577859e+12
4,1/1/2020 0:12,1/1/2020 0:14,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59.0,41.8675,-87.6322,41.8672,-87.6260,1.577859e+12
5,1/1/2020 0:19,1/1/2020 0:26,Clark St & Leland Ave,326,Southport Ave & Irving Park Rd,318.0,41.9671,-87.6674,41.9542,-87.6644,1.577860e+12


In [7]:
data = df.to_dict('records')

In [10]:
pendulum.week_starts_at(pendulum.SUNDAY)
pendulum.week_ends_at(pendulum.SATURDAY)
meta_df = pd.DataFrame(columns=['start', 'end', 'records'])
path='C:\\Users\\kaush\\Documents\\HTML\\divvy_viz_leaflet\\src\\data\\data_'
# path=''
newSet={"start":[]}
time= pendulum.from_format("1/1/2020 00:00", "MM/DD/YYYY HH:mm",tz='America/Chicago') 
counter =0
while len(newSet.keys()) !=0:
    start = time.start_of('week').timestamp()*1000
    end = time.end_of('week').timestamp()*1000
    data = df[(df['start'] >= start) & (df['start'] <= end)]
    data = data.to_dict('records')
    newSet={}
    print(len(data),start,counter)
    for i in data:
        timeStamp=int(i['start'])
        try:
            del i['start']
            newSet[timeStamp].append(i)
        except:
            newSet[timeStamp]=[i]
    if(len(data) >0):
        with open(path+str(int(start))+'.json', 'w', encoding='utf-8') as f:
            json.dump(newSet, f, ensure_ascii=False, indent=4)
        meta_df.loc[counter] = [start,end,len(data)]
    time = time.add(weeks=1)
    counter+=1


17611 1577599200000.0 0
36121 1578204000000.0 1
32168 1578808800000.0 2
23376 1579413600000.0 3
36999 1580018400000.0 4
35800 1580623200000.0 5
27164 1581228000000.0 6
34525 1581832800000.0 7
37469 1582437600000.0 8
52065 1583042400000.0 9
48239 1583647200000.0 10
18770 1584248400000.0 11
15343 1584853200000.0 12
4970 1585458000000.0 13
0 1586062800000.0 14


In [40]:
meta_df['records'].sum()

420620.0

In [37]:
# meta_df['start'] = pd.to_datetime(meta_df['start'],unit='ms')
# meta_df['end'] = pd.to_datetime(meta_df['end'],unit='ms')
meta_df

,start,end,records
0,1.577599e+12,1.578204e+12,17725.0
1,1.578204e+12,1.578809e+12,36050.0
2,1.578809e+12,1.579414e+12,32159.0
3,1.579414e+12,1.580018e+12,23459.0
4,1.580018e+12,1.580623e+12,37009.0
5,1.580623e+12,1.581228e+12,35800.0
6,1.581228e+12,1.581833e+12,27152.0
7,1.581833e+12,1.582438e+12,34574.0
8,1.582438e+12,1.583042e+12,37423.0
9,1.583042e+12,1.583647e+12,52108.0


In [38]:
time= pendulum.from_format("1/1/2020 00:00", "MM/DD/YYYY HH:mm",tz='America/Chicago') 
start = time.start_of('week').timestamp()*1000
start

1577599200000.0